# Training and Testing

**DIVE into Deep Learning**
___

In [ ]:
%load_ext tensorboard
%matplotlib inline
from util import *

## Optimization Theory

**Why can we learn from examples?**$\def\abs#1{\left\lvert #1 \right\rvert}
\def\Set#1{\left\{ #1 \right\}}
\def\mc#1{\mathcal{#1}}
\def\M#1{\boldsymbol{#1}}
\def\R#1{\mathsf{#1}}
\def\RM#1{\boldsymbol{\mathsf{#1}}}
\def\op#1{\operatorname{#1}}
\def\E{\op{E}}
\def\d{\mathrm{\mathstrut d}}
$

For the problem to be meaningful, $(\RM{x},\R{y})$ is assumed to be random with some unknown joint distribution $p_{\RM{x},\R{y}}$:

- If we always had $\R{y}=y$ instead, then a perfect classifier can just return $y$ without even looking at $\RM{x}$.

- If $p_{\RM{x},\R{y}}$ were known instead, then $p_{\R{y}|\RM{x}}$ would also be known and therefore needed not be estimated.

Examples are often collected randomly and independently from a population, i.e., as [i.i.d. samples](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables) of $(\RM{x},\R{y})$:

$$(\RM{x}_1,\R{y}_1), (\RM{x}_2,\R{y}_2), \dots\sim_{\text{i.i.d.}} p_{\RM{x},\R{y}}.$$

- If all the examples were the same instead, the examples could not show the pattern of how $\R{y}$ depends on $\RM{x}$.

- The observed distribution of i.i.d. samples converge to the unknown distribution by the [law of large number](https://en.wikipedia.org/wiki/Law_of_large_numbers).

**How to determine if a classifier is good?**

Ultimately, we desire a classifier with the maximum accuracy in predicting $\R{y}$ but doing so is [computationally too difficult](https://en.wikipedia.org/wiki/Loss_functions_for_classification).

Instead, we regard a classification algorithm to be reasonably good if 
- it can achieve the maximum possible accuracy 
- as the number of training samples goes to $\infty$.

**Definition** A probabilistic classifier for the input feature $\RM{x}$ and label $\R{y}$ with unknown joint distribution  is a conditional pmf

$$
\R{q}_{\R{y}|\RM{x}}(y|\RM{x})\qquad \text{for }\M{x}\in \mc{X}, y\in \mc{Y}
$$

defined as a function of the i.i.d. samples

$$\{(\RM{x}_i,\R{y}_i)\}_{i=1}^N$$

of $(\RM{x},\R{y})$ (but independent of $(\RM{x},\R{y})$). 
The classifier is said to be a *consistent* estimate (of $p_{\R{y}|\M{x}}$) if

$$
\begin{align}
\lim_{N\to \infty} \Pr\Set{\R{q}_{\R{y}|\RM{x}}(y|\RM{x})=p_{\R{y}|\RM{x}}(y|\RM{x})\text{ for all } y\in \mc{Y}}=1,\tag{consistency}
\end{align}
$$

namely, $\R{q}_{\R{y}|\RM{x}}(y|\RM{x})$ converges almost surely (a.s.) to $p_{\R{y}|\RM{x}}$. $\square$

A consistent probabilistic classifier gives rise to an asymptotically optimal hard-decision classifier that achieves the maximum accuracy.

**Proposition** If for some $\epsilon\geq 0$ that

$$\Pr\Set{\R{q}_{\R{y}|\RM{x}}(y|\RM{x})=p_{\R{y}|\RM{x}}(y|\RM{x}) \text{ for all } y\in \mc{Y}}=1-\epsilon,$$

the (hard-decision) classifier

$$\begin{align}\R{f}(\RM{x}):=\arg\max_{y\in \mc{Y}} \R{q}_{\R{y}|\RM{x}}(y|\RM{x})\tag{hard decision}\end{align}$$

achieves an accuracy

$$
\begin{align}
\sup_{f:\mc{X}\to \mc{Y}} \Pr(\R{y}= f(\RM{x})) &\geq \E\left[\max_{y\in \mc{Y}} p_{\R{y}|\M{x}}(y|\RM{x})\right] - \epsilon,\tag{accuracy lower bound}
\end{align}
$$

where the expectation is the maximum possible accuracy. $\square$

*Proof:* For any classifier $f$,

$$ \begin{align*}
\Pr(\R{y}= f(\RM{x}))
&= \E[p_{\R{y}|\M{x}}(f(\RM{x})|\RM{x})] \\
&\leq \E\left[\max_{y\in \mc{Y}} p_{\R{y}|\M{x}}(y|\RM{x})\right]\end{align*}
$$

where the last inequality is achievable with equality with the hard-decision classifier and $\R{q}$ replaced by $p$. This implies the desired accuracy lower bound for the case $\epsilon=0$. The more general case with $\epsilon\geq 0$ can be derived similarly. $\blacksquare$

**How can we obtain a consistent classifier?**

We train a neural network to minimize certain *loss* function.

A common loss function for classification uses the [cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) measure in information theory.

The theoretical underpinning is the following identity that relates three information quantities:

$$
\begin{align}
\overbrace{\E\left[\log \frac{1}{q_{\R{y}|\RM{x}}(\R{y}|\RM{x})}\right]}^{ \text{Cross entropy}\\ H(p_{\R{y}|\RM{x}}\|q_{\R{y}|\RM{x}}|p_{\RM{x}}):=} &\equiv \overbrace{\E\left[\log \frac{1}{p_{\R{y}|\RM{x}}(\R{y}|\RM{x})}\right]}^{\text{Conditional entropy}\\ H(\R{y}|\RM{x}):=} + \overbrace{\E\left[\log \frac{p_{\R{y}|\RM{x}}(\R{y}|\RM{x})}{q_{\R{y}|\RM{x}}(\R{y}|\RM{x})}\right].}^{\text{Divergence}\\ D(p_{\R{y}|\RM{x}}\|q_{\R{y}|\RM{x}}|p_{\RM{x}}):=}\tag{identity}
\end{align}
$$

The identity can be proved quite easily using the linearity of expectation 

$$ \E[\R{u}+\R{v}]=\E[\R{u}]+\E[\R{v}],$$

and a property of logarithm that 

$$\log uv = \log u+ \log v.$$

In [ ]:
%%html
<b>Information Identity</b><br>
<video width="100%" height="450" controls>
      <source src="https://www.cs.cityu.edu.hk/~ccha23/dl/InformationIdentity.mp4" type="video/mp4">
</video>

**Proposition** With $q_{\R{y}|\RM{x}}(y|\M{x})$ being a valid pmf of a random variable taking values from $\mc{Y}$ conditioned on a random variable taking values from $\mc{X}$, 

$$\begin{align}
\min_{q_{\R{y}|\RM{x}}} H(p_{\R{y}|\RM{x}}\|q_{\R{y}|\RM{x}}|p_{\RM{x}})
&= H(\R{y}|\RM{x})
\end{align}$$

and the optimal solution equals $p_{\R{y}|\RM{x}}(y|\RM{x})$ a.s. for all $y\in \mc{Y}$. $\square$ 

Hence, a neural network that minimizes the cross entropy equals $p_{\R{y}|\RM{x}}(y|\RM{x})$ a.s. for all $y\in \mc{Y}$ and any possible input image $\RM{x}$.

**Exercise** Prove the above proposition using the information identity and the property of divergence that 

$$\begin{align}
D(p_{\R{y}|\RM{x}}\|q_{\R{y}|\RM{x}}|p_{\RM{x}})\geq 0 \tag{positivity of divergence}
\end{align}$$

with equality iff $q_{\R{y}|\RM{x}}(y|\RM{x})=p_{\R{y}|\RM{x}}(y|\RM{x})$ a.s. (This, in turn, can be proved using the [log-sum inequality](https://en.wikipedia.org/wiki/Log_sum_inequality):

$$\begin{align}
\sum_{i} a_i \log\left(\frac{a_i}{b_i}\right) \geq (\textstyle \sum_{i} a_i) \log\left(\frac{\sum_{i} a_i}{\sum_{i} b_i}\right)\tag{log-sum inequality}
\end{align}$$ 

for any sequences $\{a_i\}$, $\{b_i\}$, and $\{c_i\}$.)

YOUR ANSWER HERE

The cross entropy cannot be computed exactly without knowing the joint distribution $p_{\RM{x}\R{y}}$. Nevertheless, it can be estimated from a batch of i.i.d. samples $(\RM{x}_{\R{j}_i},\R{y}_{\R{j}_i})$ for $1\leq i\leq n$:

$$\begin{align}
\R{L}(\theta)&:=\frac1n \sum_{i=1}^n \log \frac{1}{q_{\R{y}|\RM{x}}(\R{y}_{\R{j}_i}|\RM{x}_{\R{j}_i})}\tag{empirical loss}
\end{align}$$

where 

$$\theta := \operatorname{flat}(\M{W}^{(\ell)},\M{b}^{(\ell)}\mid 0\leq \ell \leq L)$$

is the vector of parameters of the neural network defined in (net).

A mini-batch [gradient descent algorithm](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) is often used to reduce the loss. It iteratively updates/trains the neural network parameters:

$$\theta \leftarrow \theta -s\nabla \R{L}(\theta)$$

by computing the gradient $\nabla \R{L}(\theta)$ on a randomly selected minibatch of examples and choosing an appropriate learning rate $s$.

In [ ]:
%%html
<div>
<b>What is gradient descent?</b><br>
<iframe width="100%" height="450" src="https://www.youtube.com/embed/IHZwWFHWa-w?start=468&end=510" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>
<div>
<b>How to choose the step size?</b><br>
<iframe width="100%" height="450" src="https://www.youtube.com/embed/IHZwWFHWa-w?start=403&end=415" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
</div>

- The gradient can be computed systematically using a technique called *backpropagation* due to the structure of the neural network in (net).
- The learning rate can affect the convergence rate of the loss to a local minima: 
    - $\theta$ may overshoot its optimal value if $s$ is too large, and
    - the convergence can be very slow if $\theta$ is too small.

A more advanced method called [Adam (Adaptive Momentum Estimation)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam) can adaptively choose $s$ to speed up the convergence.

## Training

The [loss function](https://keras.io/api/losses/), gradient descent algorithm, and the performance metrics can be specified using the [`compile` method](https://keras.io/api/losses/).

In [ ]:
def compile_model(model):
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(0.001),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

compile_model(model)
model.loss, model.optimizer

We can train the neural network using the method [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) of the compiled model:

In [ ]:
if input('Train? [Y/n]').lower() != 'n':
    model.fit(ds_b["train"])

**Exercise** By default, the neural network is trained for 1 epoch. What happens to the training accuracy if you rerun the above cell to train the model for another epoch?

YOUR ANSWER HERE

We can set the parameter `epochs` to train the neural network for multiple epochs since it is quite unlikely to train a neural network well with just one epoch.

To determine whether the neural network is well-trained (when to stop training), we should also use a separate validation set to evaluate the performance of the neural network. The validation set can be specified using the parameter `validation_set` as follows:

In [ ]:
if input('Train? [Y/n]').lower() != 'n':
    model.fit(ds_b["train"], epochs=6, validation_data=ds_b["test"])

**Exercise** Is the maximum validation accuracy `val_sparse_categorical_accuracy` (over different epoches) an unbiased estimate of the performance of deep learning for the given dataset? If not, how to avoid the bias?

YOUR ANSWER HERE

## Deployment

Once you are satisfied with the result, you can deploy the model as a web application.

The `mnist` folder contain the webpage `index.html` that
- presents an HTML5 canvas for users to input a handwritten digit,
- loads a trained model using [`tensorflow.js`](https://www.tensorflow.org/js),
- passes the handwritten digit to the model to predict the distribution of the digit types, and
- display the most likely digit type.

In [ ]:
display.Code('mnist/index.html')

First, save the model as follows in [HDF5](https://www.h5py.org/) format:

In [ ]:
if input('Execute? [Y/n]').lower() != 'n':
    model.save('mnist/model.h5')

Then, convert the model to files that can be loaded by [`tensorflow.js`](https://www.tensorflow.org/js):

In [ ]:
if input('Execute? [Y/n]').lower() != 'n':
    !tensorflowjs_converter --input_format keras 'mnist/model.h5' 'mnist/model'

To host the web application, run the following command:

In [ ]:
if input('Execute? [Y/n]').lower() != 'n':
    !cp -rf mnist ~/www/
    !zip -r mnist.zip mnist/*

- Preview you web application at [\<JUPYTERHUB_URL\>/user-redirect/www/mnist/](/user-redirect/www/mnist/).
- Download [`mnist.zip`](./mnist.zip) and extract it to a static web server of your choice.